In [26]:
import pydeck as pdk
import pandas as pd
import json
import ipywidgets as widgets
import os
import requests
from IPython.display import clear_output

# MAPBOX_API_KEY = "pk.eyJ1IjoidGN5Y2xvbmUiLCJhIjoiY2wwYWx5aW14MG9pdjNjbW8zdmE0dGJkNCJ9.u7D4yFZRHo0PIT45jRuQsA"

BUILDING_LOCATION_DATA = "https://raw.githubusercontent.com/TCyclone/COSC3000/main/Locations.csv"
BUILDING_LOAD_DATA = "https://raw.githubusercontent.com/TCyclone/PowerData/main/"

dfLoad = pd.DataFrame(columns=['Time'])

time = []
start = 2400
for i in range(96):
    if i%4 == 0:
        start = start - 55
        time.append(start)
    else:
        start = start - 15
        time.append(start)
        
dfLoad['Time'] = time

# put branch and node results into dataframe
dfLocation = pd.read_csv(BUILDING_LOCATION_DATA)

files = ['BioEngineeringPV.csv', 'BSB_BuildingPV.csv', 'BSB_LibraryPV.csv', 'CenterHall.csv', 'CSC_BuildingPV.csv', 'CUP_PV.csv', 'EastCampus.csv', 'EBU2_A_PV.csv', 'EBU2_B_PV.csv', 'ElectricShopPV.csv', 'GalbraithHall.csv', 'GarageFleetsPV.csv', 'GeiselLibrary.csv', 'Gilman.csv', 'GilmanParkingPV.csv', 'Hopkins.csv', 'HopkinsParkingPV.csv', 'KeelingA_PV.csv', 'KeelingB_PV.csv', 'KyoceraSkylinePV.csv', 'LeichtagPV.csv', 'Mandeville.csv', 'MayerHallPV.csv', 'MESOM_PV.csv', 'MusicBuilding.csv', 'OslerParkingPV.csv', 'OttersonHall.csv', 'PepperCanyon.csv', 'PoliceDepartment.csv', 'PowellPV.csv', 'PriceCenterA_PV.csv', 'PriceCenterB_PV.csv', 'Processed.csv', 'ProcessedData.csv', 'RadyHall.csv', 'RobinsonHall.csv', 'SDSC_PV.csv', 'SME_SolarPV.csv', 'SocialScience.csv', 'StephenBirchPV.csv', 'StudentServices.csv', 'TradeStreetPV.csv', 'BatteryStorage.csv']
for filename in files:
    power = []
    i = 0
    df = pd.read_csv(os.path.join(BUILDING_LOAD_DATA, filename))
    for i in range(96):
        power.append(df.at[i,'RealPower'])
    dfLoad[filename] = power

powers = []
coloursR = []
coloursG = []
coloursB = []
for i in range(96):
    for j in dfLocation['Location']:
        powers.append(dfLoad.at[i,j])
    dfLocation["time_" + str(dfLoad.at[i,'Time'])] = powers
    powers = []
    
for j in dfLocation['Location']:
    if j[-5].islower():
        coloursR.append(255)
        coloursG.append(0)
    else:
        coloursG.append(255)
        coloursR.append(0)
dfLocation["coloursR"] = coloursR
dfLocation["coloursG"] = coloursG

caption = widgets.Label(value='Initital time')

timeWid = widgets.SelectionSlider(
    options = ['2345', '2330', '2315', '2300', '2245', '2230', '2215', '2200', '2145', '2130', '2115', '2100', '2045', '2030', '2015', '2000', '1945', '1930', '1915', '1900', '1845', '1830', '1815', '1800', '1745', '1730', '1715', '1700', '1645', '1630', '1615', '1600', '1545', '1530', '1515', '1500', '1445', '1430', '1415', '1400', '1345', '1330', '1315', '1300', '1245', '1230', '1215', '1200', '1145', '1130', '1115', '1100', '1045', '1030', '1015', '1000', '945', '930', '915', '900', '845', '830', '815', '800', '745', '730', '715', '700', '645', '630', '615', '600', '545', '530', '515', '500', '445', '430', '415', '400', '345', '330', '315', '300', '245', '230', '215', '200', '145', '130', '115', '100', '45', '30', '15'],
    description='Time:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def on_drag_end(change):
    clear_output(wait=True)
    display(timeWid, caption)
    newTime = "time_" + str(change['new'])
    times = []
    for i in range(41):
        times.append(dfLocation.at[i,newTime])
    dfLocation["Time"] = times
    caption.value = 'Showing profile at time (in 24hr format): ' + str(change['new'])
    r = make_renderer(dfLocation)
#     display(r.show())
    r.to_html("column_layer.html")

timeWid.observe(on_drag_end, names='value')
display(timeWid, caption)

def make_renderer(dfLocation):
    """Creates the pydeck visualization for rendering"""
    view = pdk.ViewState(latitude=32.87788317, longitude=-117.2372732, bearing=45, pitch=50, zoom=15,)

    view.pitch = 75
    view.bearing = 60

    column_layer = pdk.Layer(
        "ColumnLayer",
        data=dfLocation,
        get_position=["lng", "lat"],
        get_elevation="Time",
        elevation_scale=1,
        radius=20,
        get_fill_color=["coloursR", "coloursG", "0", 200],
        pickable=True,
        auto_highlight=True,
    )

    tooltip = {
        "html": "<b>{Location}</b>, load/generation: <b>{Time} MW</b> ",
        "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
    }
    return pdk.Deck(column_layer, initial_view_state=view, tooltip=tooltip, map_provider="mapbox", map_style=pdk.map_styles.SATELLITE)


r = None

SelectionSlider(continuous_update=False, description='Time:', index=35, options=('2345', '2330', '2315', '2300…

Label(value='Showing profile at time (in 24hr format): 845')

In [23]:
import pydeck as pdk
import pandas as pd
import json
import ipywidgets as widgets
import os
import requests

BUILDING_LOCATION_DATA = "https://raw.githubusercontent.com/TCyclone/COSC3000/main/LocationsBatt.csv"
BUILDING_LOAD_DATA = "https://raw.githubusercontent.com/TCyclone/PowerData/main/"

dfLoad = pd.DataFrame(columns=['Time'])

time = []
start = 2400
for i in range(96):
    if i%4 == 0:
        start = start - 55
        time.append(start)
    else:
        start = start - 15
        time.append(start)
        
dfLoad['Time'] = time

# put branch and node results into dataframe
dfLocation = pd.read_csv(BUILDING_LOCATION_DATA)

files = ['BioEngineeringPV.csv', 'BSB_BuildingPV.csv', 'BSB_LibraryPV.csv', 'CenterHall.csv', 'CSC_BuildingPV.csv', 'CUP_PV.csv', 'EastCampus.csv', 'EBU2_A_PV.csv', 'EBU2_B_PV.csv', 'ElectricShopPV.csv', 'GalbraithHall.csv', 'GarageFleetsPV.csv', 'GeiselLibrary.csv', 'Gilman.csv', 'GilmanParkingPV.csv', 'Hopkins.csv', 'HopkinsParkingPV.csv', 'KeelingA_PV.csv', 'KeelingB_PV.csv', 'KyoceraSkylinePV.csv', 'LeichtagPV.csv', 'Mandeville.csv', 'MayerHallPV.csv', 'MESOM_PV.csv', 'MusicBuilding.csv', 'OslerParkingPV.csv', 'OttersonHall.csv', 'PepperCanyon.csv', 'PoliceDepartment.csv', 'PowellPV.csv', 'PriceCenterA_PV.csv', 'PriceCenterB_PV.csv', 'Processed.csv', 'ProcessedData.csv', 'RadyHall.csv', 'RobinsonHall.csv', 'SDSC_PV.csv', 'SME_SolarPV.csv', 'SocialScience.csv', 'StephenBirchPV.csv', 'StudentServices.csv', 'TradeStreetPV.csv', 'BatteryStorage.csv', 'NetDemand.csv']
for filename in files:
    power = []
    i = 0
    df = pd.read_csv(os.path.join(BUILDING_LOAD_DATA, filename))
    for i in range(96):
        power.append(df.at[i,'RealPower'])
    dfLoad[filename] = power

powers = []
coloursR = []
coloursG = []
coloursB = []
for i in range(96):
    for j in dfLocation['Location']:
        powers.append(dfLoad.at[i,j])
    dfLocation["time_" + str(dfLoad.at[i,'Time'])] = powers
    powers = []
    
for j in dfLocation['Location']:
    if j[-5].islower():
        coloursR.append(255)
        coloursG.append(0)
    else:
        coloursG.append(255)
        coloursR.append(0)
dfLocation["coloursR"] = coloursR
dfLocation["coloursG"] = coloursG

GREEN_RGB = [0, 255, 0, 200]
RED_RGB = [255, 0, 0, 200]

caption = widgets.Label(value='Initital time')

timeWid = widgets.SelectionSlider(
    options = ['2345', '2330', '2315', '2300', '2245', '2230', '2215', '2200', '2145', '2130', '2115', '2100', '2045', '2030', '2015', '2000', '1945', '1930', '1915', '1900', '1845', '1830', '1815', '1800', '1745', '1730', '1715', '1700', '1645', '1630', '1615', '1600', '1545', '1530', '1515', '1500', '1445', '1430', '1415', '1400', '1345', '1330', '1315', '1300', '1245', '1230', '1215', '1200', '1145', '1130', '1115', '1100', '1045', '1030', '1015', '1000', '945', '930', '915', '900', '845', '830', '815', '800', '745', '730', '715', '700', '645', '630', '615', '600', '545', '530', '515', '500', '445', '430', '415', '400', '345', '330', '315', '300', '245', '230', '215', '200', '145', '130', '115', '100', '45', '30', '15'],
    description='Time:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def on_drag_end(change, notebook_display = False):
    clear_output(wait=True)
    display(timeWid, caption)
    newTime = "time_" + str(change['new'])
    times = []
    for i in range(42):
        times.append(dfLocation.at[i,newTime])
    dfLocation["Time"] = times
    caption.value = 'Showing profile at time (in 24hr format): ' + str(change['new'])
    r = make_renderer(dfLocation)
    display(r.show())
    r.to_html("arc_layer.html")
    
timeWid.observe(on_drag_end, names='value')
display(timeWid, caption)

def make_renderer(dfLocation):
    """Creates the pydeck visualization for rendering"""
    view = pdk.ViewState(latitude=32.87788317, longitude=-117.2372732, bearing=45, pitch=50, zoom=15,)

    arc_layer = pdk.Layer(
        "ArcLayer",
        data=dfLocation,
        get_width="time_1300/12",
        get_source_position=["lngSource", "latSource"],
        get_target_position=["lngTarget", "latTarget"],
        get_tilt=15,
        get_source_color=GREEN_RGB,
        get_target_color=RED_RGB,
        pickable=True,
        auto_highlight=True,
    )

    tooltip = {
        "html": "<b>{Location}</b>, load/generation: <b>{Time} MW</b> ",
        "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
    }
    return pdk.Deck(arc_layer, initial_view_state=view, tooltip=tooltip)


r = None

KeyboardInterrupt: 